### IMPORTING THE REQUIRED LIBRARIES:

In [49]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import sportsipy
import sportsreference
from sportsipy.nfl import boxscore,constants,nfl_utils,player,schedule,roster,teams
from sportsipy.nfl.boxscore import Boxscores, Boxscore
import time
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import re
import json

#### Importing the datasets:


In [24]:
game_df=pd.read_csv('game_df.csv')
game_stats=pd.read_csv('game_stats.csv',parse_dates=['date'])

#### Understanding the data:

In [25]:
game_df.shape

(552, 14)

In [26]:
game_stats.shape

(552, 73)

In [27]:
game_df.columns

Index(['Unnamed: 0', 'boxscore', 'away_name', 'away_abbr', 'away_score',
       'home_name', 'home_abbr', 'home_score', 'winning_name', 'winning_abbr',
       'losing_name', 'losing_abbr', 'week', 'year'],
      dtype='object')

In [28]:
game_stats.columns

Index(['Unnamed: 0', 'uri', 'date', 'time', 'stadium', 'attendance',
       'duration', 'away_name', 'home_name', 'winner', 'winning_name',
       'winning_abbr', 'losing_name', 'losing_abbr', 'summary', 'won_toss',
       'roof', 'surface', 'weather', 'vegas_line', 'over_under', 'away_points',
       'away_first_downs', 'away_rush_attempts', 'away_rush_yards',
       'away_rush_touchdowns', 'away_pass_completions', 'away_pass_attempts',
       'away_pass_yards', 'away_pass_touchdowns', 'away_interceptions',
       'away_times_sacked', 'away_yards_lost_from_sacks',
       'away_net_pass_yards', 'away_total_yards', 'away_fumbles',
       'away_fumbles_lost', 'away_turnovers', 'away_penalties',
       'away_yards_from_penalties', 'away_third_down_conversions',
       'away_third_down_attempts', 'away_fourth_down_conversions',
       'away_fourth_down_attempts', 'away_time_of_possession', 'home_points',
       'home_first_downs', 'home_rush_attempts', 'home_rush_yards',
       'home_rush_

In [29]:
### Descriptive stats:

pd.DataFrame(game_df.describe(include='all')).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,552.0,NaN,NaN,NaN,275.5,159.492947,0.0,137.75,275.5,413.25,551.0
boxscore,552,552,202001190sfo,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
away_name,552,34,Seattle Seahawks,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
away_abbr,552,32,oti,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
away_score,552.0,NaN,NaN,NaN,23.76087,10.244952,0.0,17.0,24.0,31.0,59.0
home_name,552,34,Kansas City Chiefs,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_abbr,552,32,kan,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_score,552.0,NaN,NaN,NaN,23.855072,9.800589,0.0,17.0,24.0,30.25,56.0
winning_name,550,34,Kansas City Chiefs,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winning_abbr,550,32,kan,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
pd.DataFrame(game_stats.describe(include='all')).T

<ipython-input-30-2344209c7184>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  pd.DataFrame(game_stats.describe(include='all')).T


,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
Unnamed: 0,552.0,NaN,NaN,NaN,NaT,NaT,275.5,159.492947,0.0,137.75,275.5,413.25,551.0
uri,552,552,202001190sfo,1,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date,552,116,2019-12-29 00:00:00,16,2019-09-05,2021-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
time,552,23,1:00pm,277,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
stadium,552,39,MetLife Stadium,33,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
attendance,401.0,NaN,NaN,NaN,NaT,NaT,50287.024938,27100.521429,748.0,15926.0,62633.0,71029.0,93024.0
duration,552,69,3:03,25,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
away_name,552,80,"<a href=""/teams/tam/2020.htm"" itemprop=""name"">...",12,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
home_name,552,80,"<a href=""/teams/kan/2020.htm"" itemprop=""name"">...",11,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
winner,0.0,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
#dropping columns from stats:
columns_stats_deleted=['Unnamed: 0','away_name','home_name','winner','winning_name','winning_abbr','losing_name','losing_abbr','away_players','home_players']

#weather: get degrees, get humidity, get wind speed

#### Droping unnessecary column :


In [32]:
game_stats_updated=game_stats.drop(columns_stats_deleted,axis=1).copy()

In [75]:
#summary to to columns:

game_stats_updated['away_summary']=game_stats_updated.summary.apply(lambda x : sum(json.loads(re.sub("'",'"',x))['away']))
game_stats_updated['home_summary']=game_stats_updated.summary.apply(lambda x : sum(json.loads(re.sub("'",'"',x))['home']))

In [76]:
game_stats_updated

,uri,date,time,stadium,attendance,duration,summary,won_toss,roof,surface,weather,vegas_line,over_under,away_points,away_first_downs,away_rush_attempts,away_rush_yards,away_rush_touchdowns,away_pass_completions,away_pass_attempts,away_pass_yards,away_pass_touchdowns,away_interceptions,away_times_sacked,away_yards_lost_from_sacks,away_net_pass_yards,away_total_yards,away_fumbles,away_fumbles_lost,away_turnovers,away_penalties,away_yards_from_penalties,away_third_down_conversions,away_third_down_attempts,away_fourth_down_conversions,away_fourth_down_attempts,away_time_of_possession,home_points,home_first_downs,home_rush_attempts,home_rush_yards,home_rush_touchdowns,home_pass_completions,home_pass_attempts,home_pass_yards,home_pass_touchdowns,home_interceptions,home_times_sacked,home_yards_lost_from_sacks,home_net_pass_yards,home_total_yards,home_fumbles,home_fumbles_lost,home_turnovers,home_penalties,home_yards_from_penalties,home_third_down_conversions,home_third_down_attempts,home_fourth_down_conversions,home_fourth_down_attempts,home_time_of_possession,away_abbr,home_abbr,away_summary,home_summary
0,201912080ram,2019-12-08,8:20pm,Los Angeles Memorial Coliseum,71501.0,3:01,"{'away': [3, 0, 6, 3], 'home': [7, 14, 0, 7]}",Rams (deferred),Outdoors,Grass,"60 degrees, relative humidity 79%, wind 4 mph",Los Angeles Rams -1.0,47.5 (under),12,16,21-106-0,21-106-0,21-106-0,22-36-245-0-1,22-36-245-0-1,22-36-245-0-1,22-36-245-0-1,22-36-245-0-1,5-43,5-43,202,308,2-0,2-0,1,8-64,8-64,5-14,5-14,1-2,1-2,28:56,28,26,35-162-2,35-162-2,35-162-2,22-32-293-2-2,22-32-293-2-2,22-32-293-2-2,22-32-293-2-2,22-32-293-2-2,0-0,0-0,293,455,0-0,0-0,2,3-20,3-20,7-13,7-13,0-0,0-0,31:04,SEA,LAR,12,28
1,202011220was,2020-11-22,1:00pm,FedExField,NaN,3:02,"{'away': [0, 9, 0, 0], 'home': [7, 0, 10, 3]}",Bengals (deferred),Outdoors,Grass,"57 degrees, relative humidity 63%, wind 7 mph",Washington Football Team -2.0,48.0 (under),9,18,18-70-0,18-70-0,18-70-0,25-44-233-1-1,25-44-233-1-1,25-44-233-1-1,25-44-233-1-1,25-44-233-1-1,4-31,4-31,202,272,1-1,1-1,2,3-31,3-31,3-13,3-13,0-2,0-2,29:14,20,20,34-164-1,34-164-1,34-164-1,17-25-166-1-1,17-25-166-1-1,17-25-166-1-1,17-25-166-1-1,17-25-166-1-1,2-5,2-5,161,325,1-0,1-0,1,6-49,6-49,4-13,4-13,0-0,0-0,30:46,CIN,WAS,9,20
2,202101090buf,2021-01-09,1:05pm,Bills Stadium,6772.0,3:16,"{'away': [3, 7, 0, 14], 'home': [7, 7, 3, 10]}",Bills (deferred),Outdoors,Astroturf,"34 degrees, relative humidity 59%, wind 6 mph",Buffalo Bills -7.0,50.5 (over),24,27,30-163-1,30-163-1,30-163-1,27-46-309-2-0,27-46-309-2-0,27-46-309-2-0,27-46-309-2-0,27-46-309-2-0,0-0,0-0,309,472,1-0,1-0,0,2-10,2-10,9-17,9-17,2-4,2-4,34:17,27,22,21-96-1,21-96-1,21-96-1,26-35-324-2-0,26-35-324-2-0,26-35-324-2-0,26-35-324-2-0,26-35-324-2-0,2-23,2-23,301,397,1-0,1-0,0,2-7,2-7,2-9,2-9,0-0,0-0,25:43,IND,BUF,24,27
3,202101100nor,2021-01-10,4:40pm,Mercedes-Benz Superdome,3000.0,3:10,"{'away': [0, 3, 0, 6], 'home': [7, 0, 7, 7]}",Bears (deferred),Dome,Astroturf,NaN,New Orleans Saints -11.0,48.0 (under),9,11,19-48-0,19-48-0,19-48-0,19-29-199-1-0,19-29-199-1-0,19-29-199-1-0,19-29-199-1-0,19-29-199-1-0,1-8,1-8,191,239,0-0,0-0,0,9-50,9-50,1-10,1-10,0-1,0-1,21:02,21,27,35-123-1,35-123-1,35-123-1,28-39-265-2-0,28-39-265-2-0,28-39-265-2-0,28-39-265-2-0,28-39-265-2-0,1-3,1-3,262,385,2-1,2-1,1,4-20,4-20,11-17,11-17,0-1,0-1,38:58,CHI,NOR,9,21
4,202109120nyg,2021-09-12,4:25pm,MetLife Stadium,74119.0,2:53,"{'away': [0, 10, 7, 10], 'home': [0, 7, 0, 6]}",Broncos (deferred),Outdoors,Fieldturf,"82 degrees, relative humidity 56%, wind 19 mph",Denver Broncos -3.0,41.5 (under),27,24,28-165-1,28-165-1,28-165-1,28-36-264-2-0,28-36-264-2-0,28-36-264-2-0,28-36-264-2-0,28-36-264-2-0,2-9,2-9,255,420,1-1,1-1,1,5-51,5-51,7-15,7-15,3-3,3-3,35:08,13,19,20-60-1,20-60-1,20-60-1,22-37-267-1-0,22-37-267-1-0,22-37-267-1-0,22-37-267-1-0,22-37-267-1-0,2-13,2-13,254,314,1-1,1-1,1,4-48,4-48,6-12,6-12,1-3,1-3,24:52,DEN,NYG,27,13
5,201911250ram,2019-11-25,8:15pm,Los Angeles Memorial Coliseum,72409.0,2:54,"{'away': [